here, i will attempt to document a discrepancy in the speed issue between `LangevinSplittingDynamicsMove` and `LangevinIntegrator`

In [1]:
import simtk.openmm as openmm
from openmmtools.constants import kB
import simtk.unit as unit
temperature = 300 * unit.kelvin
kT = kB * temperature
beta = 1.0/kT
from openmmtools.states import ThermodynamicState, SamplerState, CompoundThermodynamicState
from openmmtools import mcmc, utils
import openmmtools.cache as cache
from perses.dispersed.utils import configure_platform
import openmmtools.integrators as integrators
from protein_test_cycle import *
from openmmtools import mcmc, utils

conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


and we should be using CUDA...

In [2]:
print(cache.global_context_cache.platform.getName())
print(cache.global_context_cache.platform.getPropertyDefaultValue('CudaPrecision'))

CUDA
mixed


In [5]:
from openmmtools.testsystems import SrcExplicit as src
src = src()

In [36]:
def create_integrator(system, topology, positions, constraint_tol):
    """
    create lambda alchemical states, thermodynamic states, sampler states, integrator, and return context, thermostate, sampler_state, integrator
    """
    
    fast_thermodynamic_state = ThermodynamicState(system, temperature = temperature)
    
    fast_sampler_state = SamplerState(positions = positions, box_vectors = system.getDefaultPeriodicBoxVectors())
    
    mcmc_moves=mcmc.LangevinSplittingDynamicsMove(timestep = 4.0 * unit.femtoseconds,
                                                             collision_rate=5.0 / unit.picosecond,
                                                             n_steps=250,
                                                             reassign_velocities=False,
                                                             n_restart_attempts=20,
                                                             splitting="V R O R V",
                                                             constraint_tolerance=constraint_tol)
    
    return mcmc_moves, fast_thermodynamic_state, fast_sampler_state


    
    
    
    

In [37]:
def create_langevin_integrator(system, topology, positions, constraint_tol):
    """
    create lambda alchemical states, thermodynamic states, sampler states, integrator, and return context, thermostate, sampler_state, integrator
    """

        
    fast_thermodynamic_state = ThermodynamicState(system, temperature = temperature)
    
    fast_sampler_state = SamplerState(positions = positions, box_vectors = system.getDefaultPeriodicBoxVectors())\
    
    integrator_1 = integrators.LangevinIntegrator(temperature = temperature,
                                                     timestep = 4.0* unit.femtoseconds,
                                                     splitting = 'V R O R V',
                                                     measure_shadow_work = False,
                                                     measure_heat = False,
                                                     constraint_tolerance = constraint_tol,
                                                     collision_rate = 5.0 / unit.picoseconds)

    
    
    #print(integrator_1.getConstraintTolerance())
    
    fast_context, fast_integrator = cache.global_context_cache.get_context(fast_thermodynamic_state, integrator_1)
    
    
    fast_sampler_state.apply_to_context(fast_context)
    
    return fast_context, fast_thermodynamic_state, fast_sampler_state, fast_integrator

In [38]:
def time_integrator(move, thermostate, sstate, num_steps = 10):
    import time
    _time = []
    #integrator.step(1)
    move.apply(thermostate, sstate)
    for i in range(num_steps):
        start = time.time()
        #integrator.step(1)
        move.apply(thermostate, sstate)
        end = time.time() - start
        _time.append(end)
    return np.array(_time)

In [39]:
def time_lan_integrator(integrator, num_steps = 10):
    import time
    _time = []
    integrator.step(1)
    #move.apply(thermostate, sstate)
    for i in range(num_steps):
        start = time.time()
        integrator.step(1)
        #move.apply(thermostate, sstate)
        end = time.time() - start
        _time.append(end)
    return np.array(_time)

let's compare the speeds of all the atom maps right now...

In [40]:
from openmmtools.utils import get_available_platforms

In [41]:
plats = get_available_platforms()
for plat in plats:
    print(plat.getName())

Reference
CPU
CUDA
OpenCL


In [42]:
print(f"this is for platform {cache.global_context_cache.platform.getName()}")
_, _, _, integrator = create_langevin_integrator(src.system, src.topology, src.positions, 1e-6)
fast = time_lan_integrator(integrator, 250)    

mcmc_moves, thermostate, sstate = create_integrator(src.system, src.topology, src.positions, 1e-6)
slow = time_integrator(mcmc_moves, thermostate, sstate, 1)
    

this is for platform CUDA


In [44]:
print(np.average(fast), np.std(fast), np.average(slow)/250, np.std(slow))


0.011655369758605956 0.0062754618314890855 0.01177958583831787 0.0


so the mcmc move is about 4x slower than if we simply create a `LangevinIntegrator`

now , we'll just serialize the system and state